 **IMPORTING AND SETTING**

In [5]:
import torch
from torch import nn
from torch.nn import functional as F
from torchinfo import summary
import nltk
from nltk.stem import PorterStemmer
import tqdm
import json
import string
from torch.utils.data import DataLoader, Dataset
import random

**PRINTING OUT MY DATASET**

In [4]:
with open('Ecommerce_FAQ_intents.json', 'r') as f:
    data = json.load(f)
data

{'intents': [{'tag': 'create_account',
   'patterns': ['How can I create an account?',
    'How do I sign up?',
    'I want to register',
    'Can I make a new account?',
    'Sign up process',
    'Register on your website'],
   'responses': ["To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."],
   'context_set': ''},
  {'tag': 'payment_methods',
   'patterns': ['What payment methods do you accept?',
    'How can I pay?',
    'Do you take credit cards?',
    'Can I use PayPal?',
    'Payment options',
    'Accepted payment types'],
   'responses': ['We accept major credit cards, debit cards, and PayPal as payment methods for online orders.'],
   'context_set': ''},
  {'tag': 'track_order',
   'patterns': ['How can I track my order?',
    'Where is my order?',
    'Order tracking',
    'Track shipment',
    'Check order status'],
   'responses': ["You can track your order by loggi

**Setting up the tokenization and stem function**

In [6]:
stemmer = PorterStemmer()

def tokenize(text: str):
    return nltk.tokenize.word_tokenize(text)

def stem(input: list):
    return stemmer.stem(input)

In [10]:
# Testing stem and tokenize function
[stem(i.lower()) for i in tokenize("We are out of stocks")]

['we', 'are', 'out', 'of', 'stock']

In [11]:
allwords = []
tags = []
xy = []

for intents in data['intents']:
    if intents['tag'] not in tags:
        tag = intents['tag']
        tags.append(tag)

    for patterns in intents['patterns']:
        word = tokenize(patterns.lower())
        word = [stem(x) for x in word if x not in list(string.punctuation)]
        allwords.extend(word)
        xy.append(([i for i in word], tag))

print(xy)
allwords = sorted(set(allwords))
allwords

[(['how', 'can', 'i', 'creat', 'an', 'account'], 'create_account'), (['how', 'do', 'i', 'sign', 'up'], 'create_account'), (['i', 'want', 'to', 'regist'], 'create_account'), (['can', 'i', 'make', 'a', 'new', 'account'], 'create_account'), (['sign', 'up', 'process'], 'create_account'), (['regist', 'on', 'your', 'websit'], 'create_account'), (['what', 'payment', 'method', 'do', 'you', 'accept'], 'payment_methods'), (['how', 'can', 'i', 'pay'], 'payment_methods'), (['do', 'you', 'take', 'credit', 'card'], 'payment_methods'), (['can', 'i', 'use', 'paypal'], 'payment_methods'), (['payment', 'option'], 'payment_methods'), (['accept', 'payment', 'type'], 'payment_methods'), (['how', 'can', 'i', 'track', 'my', 'order'], 'track_order'), (['where', 'is', 'my', 'order'], 'track_order'), (['order', 'track'], 'track_order'), (['track', 'shipment'], 'track_order'), (['check', 'order', 'statu'], 'track_order'), (['what', 'is', 'your', 'return', 'polici'], 'return_policy'), (['can', 'i', 'return', 'my'

["'s",
 "'ve",
 'a',
 'about',
 'accept',
 'access',
 'account',
 'activ',
 'add',
 'address',
 'adjust',
 'affili',
 'after',
 'again',
 'agent',
 'alert',
 'all',
 'am',
 'amount',
 'an',
 'and',
 'ani',
 'anoth',
 'anyon',
 'appli',
 'are',
 'arriv',
 'as',
 'assist',
 'at',
 'avail',
 'b2b',
 'back',
 'balanc',
 'be',
 'been',
 'benefit',
 'better',
 'between',
 'bill',
 'bought',
 'broke',
 'broken',
 'bug',
 'bulk',
 'busi',
 'but',
 'buy',
 'by',
 'bye',
 'ca',
 'call',
 'can',
 'cancel',
 'card',
 'care',
 'career',
 'chang',
 'charg',
 'chariti',
 'chart',
 'chat',
 'check',
 'checkout',
 'claim',
 'clean',
 'click',
 'close',
 'cloth',
 'code',
 'collect',
 'comment',
 'commun',
 'compani',
 'compar',
 'competitor',
 'complet',
 'confirm',
 'contact',
 'corpor',
 'countri',
 'coupon',
 'creat',
 'creator',
 'credit',
 'csr',
 'current',
 'custom',
 'cya',
 'damag',
 'data',
 'date',
 'day',
 'deal',
 'declin',
 'delay',
 'delet',
 'deliv',
 'deliveri',
 'depart',
 'design',
 

In [12]:
def bag_of_words(allwords: list, input: list) -> torch.tensor:
    word = torch.zeros(size=[len(allwords)])
    for idx, text in enumerate(allwords):
        if text in input:
            word[idx] = 1
    return word

bag_of_words(allwords, ['is', 'anyon', 'there', 'how', 'are', 'you'])

x_train = []
y_train = []
for text, tag in xy:
    text = bag_of_words(allwords, text)
    x_train.append(text)
    y_train.append(tags.index(tag))

x_train[10], y_train[10]

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.,

In [13]:
# Building the dataset and dataloader
torch.manual_seed(1)
torch.cuda.manual_seed(1)

class ChatDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

    def __str__(self):
        return f'Data set of {self.__class__.__name__}'

dataset = ChatDataset(x=x_train, y=y_train)

chat_dataloader = DataLoader(dataset=dataset, shuffle=True, batch_size=8)
print(f'Length of dataset: {len(dataset)} | Length of dataloader: {len(chat_dataloader)}')
print(dataset[0])

Length of dataset: 473 | Length of dataloader: 60
(tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [14]:
# Hyper parameters
input_features = len(allwords)
output_features = len(tags)
hidden_units = 256

# The model

class ChatbotV1(nn.Module):
    def __init__(self, input_features, output_features, hidden_units):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=input_features, out_features=hidden_units)
        self.layer_2 = nn.Linear(in_features=hidden_units, out_features=hidden_units)
        self.layer_3 = nn.Linear(in_features=hidden_units, out_features=hidden_units)
        self.classifier = nn.Linear(in_features=hidden_units, out_features=output_features)


    def forward(self, x: torch.tensor) -> torch.tensor:
        return self.classifier(F.relu(self.layer_3(F.relu(self.layer_2(F.relu(self.layer_1(x)))))))

model_v1 = ChatbotV1(input_features=input_features, output_features=output_features, hidden_units=hidden_units)
summary(model_v1, input_size=[len(allwords)])

Layer (type:depth-idx)                   Output Shape              Param #
ChatbotV1                                [101]                     --
├─Linear: 1-1                            [256]                     104,192
├─Linear: 1-2                            [256]                     65,792
├─Linear: 1-3                            [256]                     65,792
├─Linear: 1-4                            [101]                     25,957
Total params: 261,733
Trainable params: 261,733
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 62.98
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 1.05
Estimated Total Size (MB): 1.06

In [15]:
# The optimizer and loss function
criteron = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_v1.parameters(), lr=1e-3)

In [16]:
# Training loop

epochs = 50

for epoch in tqdm.tqdm(range(epochs)):
    model_v1.train()
    train_loss = 0
    train_acc = 0
    for x, y in chat_dataloader:
        y_logits = model_v1(x)
        y_preds = torch.argmax(torch.softmax(y_logits, dim=1), dim=-1)
        loss = criteron(y_logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # calculate loss and accuracy
        train_loss += loss.item()
        train_acc += torch.eq(y_preds, y).sum() / len(y)

    train_loss = train_loss / len(chat_dataloader)
    train_acc = train_acc / len(chat_dataloader)

    model_v1.eval()
    if (epoch + 1) % 10 == 0:
        print(f'EPOCH {epoch + 1} || Loss: {train_loss:.4f} | Accuracy: {train_acc * 100:.2f}%')

 20%|██        | 10/50 [00:02<00:10,  3.72it/s]

EPOCH 10 || Loss: 0.2869 | Accuracy: 92.29%


 40%|████      | 20/50 [00:06<00:09,  3.01it/s]

EPOCH 20 || Loss: 0.0475 | Accuracy: 98.54%


 60%|██████    | 30/50 [00:10<00:10,  1.95it/s]

EPOCH 30 || Loss: 0.0334 | Accuracy: 98.33%


 80%|████████  | 40/50 [00:14<00:03,  2.65it/s]

EPOCH 40 || Loss: 0.0264 | Accuracy: 98.33%


100%|██████████| 50/50 [00:19<00:00,  2.57it/s]

EPOCH 50 || Loss: 0.2353 | Accuracy: 92.50%


In [17]:
def chat(text):
    text = tokenize(text)
    text = [stem(w) for w in text]
    text = bag_of_words(allwords, text)

    model_v1.eval()
    with torch.inference_mode():
        logit = model_v1(text)

    pred = torch.argmax(torch.softmax(logit, dim=-1))
    c_tag = tags[pred]
    print(f'Index: {pred} | Class: {c_tag}')
    res = [random.choice(x["responses"]) for x in data["intents"] if x["tag"] == c_tag][0]
    print(f'Response: {res}')
    return res


chat('Who created you')

Index: 99 | Class: creator
Response: Ebhota Walter Eromosele


'Ebhota Walter Eromosele'

In [18]:
from pathlib import Path

MODEL_SAVE_PATH = Path('model')
MODEL_SAVE_PATH.mkdir(exist_ok=True, parents=True)
MODEL_NAME = 'ChatbotV1'

In [19]:
# Saving the model
torch.save(model_v1.state_dict(), MODEL_SAVE_PATH/MODEL_NAME)

In [20]:
loaded_model = ChatbotV1(input_features, output_features, hidden_units)
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH/MODEL_NAME))

def chat(text):
    text = tokenize(text)
    text = [stem(w) for w in text]
    text = bag_of_words(allwords, text)

    loaded_model.eval()
    with torch.inference_mode():
        logit = loaded_model(text)

    pred = torch.argmax(torch.softmax(logit, dim=-1))
    c_tag = tags[pred]
    print(f'Index: {pred} | Class: {c_tag}')
    res = [random.choice(x["responses"]) for x in data["intents"] if x["tag"] == c_tag][0]
    print(f'Response: {res}')
    return res


chat('Who created you')

Index: 99 | Class: creator
Response: Ebhota Walter Eromosele


'Ebhota Walter Eromosele'